### https://radimrehurek.com/gensim/models/doc2vec.html


In [4]:
from gensim import utils
from gensim.models.doc2vec import TaggedDocument 
from gensim.models import Doc2Vec
import random
import os
import nltk.stem as stem

from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.metrics import classification_report_imbalanced
import imblearn


import gensim
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from tqdm import trange
import numpy as np
import pandas as pd


In [5]:
root_path = './data/dataset/ebay2gg_root_origin'
origin_data_tmp = [os.path.join(root_path,i) for i in os.listdir(os.path.join(root_path))]
origin_data=[]
for i in origin_data_tmp:
    if 'new' not in i and 'check' not in i:
        origin_data.append(i)

In [6]:
# for i in [20,18,14,12,8,5,4,1]:
#     print(origin_data[i])
#     origin_data.remove(origin_data[i])
    
# origin_data=origin_data[1:]
origin_data

['./data/dataset/ebay2gg_root_origin/Office Supplies.txt',
 './data/dataset/ebay2gg_root_origin/Apparel & Accessories.txt',
 './data/dataset/ebay2gg_root_origin/Vehicles & Parts.txt',
 './data/dataset/ebay2gg_root_origin/Arts & Entertainment.txt',
 './data/dataset/ebay2gg_root_origin/Food, Beverages & Tobacco.txt',
 './data/dataset/ebay2gg_root_origin/Software.txt',
 './data/dataset/ebay2gg_root_origin/Baby & Toddler.txt',
 './data/dataset/ebay2gg_root_origin/Business & Industrial.txt',
 './data/dataset/ebay2gg_root_origin/Home & Garden.txt',
 './data/dataset/ebay2gg_root_origin/Luggage & Bags.txt',
 './data/dataset/ebay2gg_root_origin/Health & Beauty.txt',
 './data/dataset/ebay2gg_root_origin/Religious & Ceremonial.txt',
 './data/dataset/ebay2gg_root_origin/Sporting Goods.txt',
 './data/dataset/ebay2gg_root_origin/Hardware.txt',
 './data/dataset/ebay2gg_root_origin/Media.txt',
 './data/dataset/ebay2gg_root_origin/Electronics.txt',
 './data/dataset/ebay2gg_root_origin/Furniture.txt',
 

In [12]:
X=[]
y=[]
label=0
for i in origin_data:
    if i=="./data/dataset/.ipynb_checkpoints":
        continue
    data = open(i).readlines()
    #print(len(data))
    for j in data:
        X.append(j)
        y.append(label)
    label+=1


In [13]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

X_train , X_test , y_train  , y_test = train_test_split(X,y,test_size=0.3,stratify =y) 


In [14]:
from pprint import pprint
pprint('Training class distributions summary: {}'.format(Counter(y_train)))
pprint('Test class distributions summary: {}'.format(Counter(y_test)))

('Training class distributions summary: Counter({3: 6841, 1: 4068, 15: 3109, '
 '12: 2536, 7: 2482, 8: 2299, 20: 1605, 10: 1173, 19: 428, 14: 325, 17: 300, '
 '5: 245, 0: 226, 13: 224, 2: 136, 4: 126, 6: 99, 16: 62, 9: 36, 18: 24, 11: '
 '22})')
('Test class distributions summary: Counter({3: 2932, 1: 1744, 15: 1333, 12: '
 '1087, 7: 1064, 8: 986, 20: 688, 10: 502, 19: 184, 14: 139, 17: 129, 5: 105, '
 '0: 97, 13: 96, 2: 58, 4: 54, 6: 42, 16: 26, 9: 15, 18: 10, 11: 9})')


In [16]:
from sklearn.ensemble import RandomForestClassifier
pipe1 = make_pipeline(TfidfVectorizer(), RandomForestClassifier(n_estimators=20,n_jobs=8))
pipe1.fit(X_train, y_train)
y_pred = pipe1.predict(X_test)

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.95      0.86      1.00      0.90      0.98      0.95        97
          1       0.97      0.98      1.00      0.98      0.98      0.97      1744
          2       0.73      0.64      1.00      0.68      0.85      0.70        58
          3       0.92      0.95      0.97      0.94      0.95      0.90      2932
          4       0.71      0.65      1.00      0.68      0.84      0.69        54
          5       0.80      0.78      1.00      0.79      0.90      0.79       105
          6       0.81      0.93      1.00      0.87      0.90      0.80        42
          7       0.89      0.88      0.99      0.89      0.94      0.88      1064
          8       0.91      0.90      0.99      0.91      0.95      0.90       986
          9       0.75      0.60      1.00      0.67      0.87      0.73        15
         10       1.00      0.97      1.00      0.98      1.00      0.99       502
   

In [139]:
#use smote to handle samples imbalanced problem

from imblearn.over_sampling import RandomOverSampler,SMOTE
from sklearn.ensemble import RandomForestClassifier


pipe = make_pipeline_imb(TfidfVectorizer(),
                         #RandomUnderSampler(),
                         SMOTE(random_state=0,n_jobs=8),
                         RandomForestClassifier(n_jobs=8))

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

          0       0.95      0.87      1.00      0.91      0.98      0.95        97
          1       0.98      0.97      1.00      0.97      0.99      0.97      1744
          2       0.61      0.71      1.00      0.66      0.78      0.59        58
          3       0.92      0.94      0.97      0.93      0.95      0.90      2932
          4       0.54      0.65      1.00      0.59      0.73      0.51        54
          5       0.84      0.81      1.00      0.83      0.92      0.83       105
          6       0.82      0.86      1.00      0.84      0.90      0.80        42
          7       0.89      0.85      0.99      0.87      0.93      0.86      1064
          8       0.90      0.91      0.99      0.90      0.94      0.88       986
          9       0.75      0.60      1.00      0.67      0.87      0.73        15
         10       0.98      0.97      1.00      0.97      0.99      0.97       502
   

In [34]:
new_x_test = ['Musical Instruments & Gear:Instruction Books, CDs & Video:Strings']
pred_label = pipe1.predict(new_x_test)
for i in pred_label:
    print(origin_data[i])

./data/dataset/ebay2gg_root_origin/Arts & Entertainment.txt


In [35]:
#test tf_idf on one root category
import pandas as pd
import nltk.stem as stem
s=stem.SnowballStemmer('english')


df3 = pd.read_csv('./data/gpc_id2name.tsv',sep='\t')
a=df3[["GPC_NAME"]].values
b=[[i[0],i[0]]    for i in a]
for i in range(len(b)):
    b[i][1] =  b[i][1].split(">")[0].strip()
    
df_101 = pd.DataFrame(b,columns=['gg_categ',"gg_first_categ"])
train_data = df_101.query("gg_first_categ=='Arts & Entertainment'")
train_data=train_data[['gg_categ']].values

In [36]:
new_arr = train_data.reshape(len(train_data))
new_list = list(new_arr)
new_new_list = [i.replace(">"," ").replace("&"," ") for i in new_list]
print("Number of documents:",len(new_new_list))
gen_docs = [[s.stem(w.lower()) for w in word_tokenize(text)] 
            for text in new_new_list]
print(gen_docs[131])
print(gen_docs[84])
print(gen_docs[161])
print(gen_docs[169])

dictionary = gensim.corpora.Dictionary(gen_docs)
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
tf_idf = gensim.models.TfidfModel(corpus)
print(len(dictionary))
sims = gensim.similarities.Similarity('./',tf_idf[corpus],num_features=len(dictionary),num_best=5)


Number of documents: 487
['art', 'entertain', 'hobbi', 'creativ', 'art', 'art', 'craft', 'art', 'craft', 'tool', 'frame', ',', 'hoop', 'stretcher']
['art', 'entertain', 'hobbi', 'creativ', 'art', 'art', 'craft', 'art', 'craft', 'materi', 'textil', 'craft', 'canva']
['art', 'entertain', 'hobbi', 'creativ', 'art', 'art', 'craft', 'craft', 'organ', 'thread', 'yarn', 'organ']
['art', 'entertain', 'hobbi', 'creativ', 'art', 'collect', 'autograph']
445


In [37]:
#查不到得处理
test_sample = new_x_test[0]
test_sample1=test_sample.replace("&"," ").replace(":"," ").replace('(',' ').replace(")"," ").replace("/"," ").replace(","," ")
query_doc = [s.stem(w.lower()) for w in word_tokenize(test_sample1)]
print(query_doc)
query_doc_bow = dictionary.doc2bow(query_doc)
print(query_doc_bow)
query_doc_tf_idf = tf_idf[query_doc_bow]
print(query_doc_tf_idf)


['music', 'instrument', 'gear', 'instruct', 'book', 'cds', 'video', 'string']
[(249, 1), (250, 1), (297, 1)]
[(249, 0.2787648854759874), (250, 0.2787648854759874), (297, 0.9190104881072464)]


In [38]:
res=sims[query_doc_tf_idf]

# print(np.argmax(res)+1,len(res))
# print("google category:   "+new_list[np.argmax(res)])
# print("ebay category  :   "+test_sample)

print(res)
for i in range(len(res)):
    print("google_category_"+str(i+1)+":   "+new_list[res[i][0]])           

# print("google_category_1:   "+new_list[res[0][0]])
# print("google_category_2:   "+new_list[res[1][0]])
# print("google_category_3:   "+new_list[res[2][0]])
# print("google_category_4:   "+new_list[res[3][0]])
# print("google_category_5:   "+new_list[res[4][0]])


print("ebay_category    :   "+test_sample)


[(412, 0.96910041570663452), (267, 0.77196121215820312), (290, 0.76630705595016479), (291, 0.76401937007904053), (294, 0.7584843635559082)]
google_category_1:   Arts & Entertainment > Hobbies & Creative Arts > Musical Instruments > String Instruments
google_category_2:   Arts & Entertainment > Hobbies & Creative Arts > Musical Instrument & Orchestra Accessories > String Instrument Accessories
google_category_3:   Arts & Entertainment > Hobbies & Creative Arts > Musical Instrument & Orchestra Accessories > String Instrument Accessories > Orchestral String Instrument Accessories > Orchestral String Instrument Strings
google_category_4:   Arts & Entertainment > Hobbies & Creative Arts > Musical Instrument & Orchestra Accessories > String Instrument Accessories > String Instrument Care & Cleaning
google_category_5:   Arts & Entertainment > Hobbies & Creative Arts > Musical Instrument & Orchestra Accessories > String Instrument Accessories > String Instrument Care & Cleaning > String Instru

In [ ]:
                    Arts & Entertainment > Hobbies & Creative Arts > Collectibles